# Dataset generation

In [7]:
#!/usr/bin/env python
# coding: utf-8

import os

import numpy as np
import pandas as pd
import torch

from scvi.dataset import GeneExpressionDataset
from scvi.inference import UnsupervisedTrainer
from scvi.models import VAE
from scvi.utils import make_dir_if_necessary


N_CELLS_QUERY = 100
FOLDER_NAME = "runs/symsim_ablation_study"
DELTA = 0.8
SEED = 0
N_CELLS_REF = 500
N_OTHERS = 2000
OFFSET = 1e-10

N_EPOCHS = 1000
N_PICKS = 1
N_SAMPLES = 10000

print("USING OFFSET = {}".format(OFFSET))

RECONSTRUCTION_LOSS = "nb"
N_PICKS = 2
ADVANCED_PROPS = True
N_SAMPLES = 5000
# N_EPOCHS = 500
N_LATENT = 50
PIN_MEMORY = False
USE_BATCH_NORM = False
PATH_TO_SCRIPTS = "MYPATHTOSCRIPTS"
DIR_PATH = "{FOLDER_NAME}/symsim_bimod_{N_CELLS_QUERY}_{N_CELLS_REF}_{OFFSET}_{DELTA}_{N_OTHERS}_{SEED}".format(
    FOLDER_NAME=FOLDER_NAME,
    N_CELLS_QUERY=N_CELLS_QUERY,
    N_CELLS_REF=N_CELLS_REF,
    OFFSET=OFFSET,
    DELTA=DELTA,
    N_OTHERS=N_OTHERS,
    SEED=SEED,
)


Q0 = 1e-1
# np.random.seed(SEED)
make_dir_if_necessary(DIR_PATH)
print(os.listdir(DIR_PATH))

#####################################################
#####################################################
symsim_data_path = (
    "./data/symsim_nevf30.n_de_evf_18.sigma_0.rep_1/DE"
)
x_obs_all = pd.read_csv(
    os.path.join(symsim_data_path, "DE_med.obsv.3.csv"), index_col=0
).T
batch_info = (
    pd.read_csv(os.path.join(symsim_data_path, "DE_med.batchid.csv"), index_col=0) - 1
)
metadata = pd.read_csv(
    os.path.join(symsim_data_path, "DE_med.cell_meta.csv"), index_col=0
)

x_obs = x_obs_all

# NEW
label_a = 1
label_b = 2

k_on = pd.read_csv(os.path.join(symsim_data_path, "DE_med.kon_mat.csv"))
k_off = pd.read_csv(os.path.join(symsim_data_path, "DE_med.koff_mat.csv"))
s_mat = pd.read_csv(os.path.join(symsim_data_path, "DE_med.s_mat.csv"))

k_on = k_on.values.T
k_off = k_off.values.T
s_mat = s_mat.values.T

# labels = dataset.labels.squeeze()
labels = metadata["pop"].values - 1
where_a = np.where(labels == 1)[0]  # [:1000]
where_b = np.where(labels == 2)[0]  # [:1000]
print(label_a, label_b, where_a.shape, where_b.shape)
means_a = s_mat[where_a] * k_on[where_a] / (k_on[where_a] + k_off[where_a])
means_b = s_mat[where_b] * k_on[where_b] / (k_on[where_b] + k_off[where_b])
lfc_dist_gt = (np.log2(means_a) - np.log2(means_b))[:, select_gene]
lfc_gt = lfc_dist_gt.mean(0)
lfc_gt_alt = np.log2(means_a.mean(0)) - np.log2(means_b.mean(0))


select_gene = np.arange(x_obs_all.values.shape[1])
non_null_genes = x_obs_all.sum(0) > 0
bimod_select = (np.abs(lfc_gt) <= 0.2) | (np.abs(lfc_gt) >= 1.0)

select_gene = non_null_genes * bimod_select
# x_obs_all = x_obs_all.loc[:, non_null_genes]

x_obs = x_obs_all.loc[:, select_gene]
true_ = pd.read_csv(
    os.path.join(symsim_data_path, "DE_med.true.csv"), index_col=0
).T.loc[:, select_gene]
lfc_info = pd.read_csv(
    os.path.join(symsim_data_path, "med_theoreticalFC.csv"), index_col=0
).loc[select_gene, :]
# lfc_info[d_key].values
print("Original data distrib: ", metadata["pop"].groupby(metadata["pop"]).size())

# Data only
# n_examples_ct = np.array([2000, 2000, 2000, 2000, 2000])
n_examples_ct = np.array([N_OTHERS, N_CELLS_REF, N_CELLS_QUERY, N_OTHERS, N_OTHERS])

selected_indices = (
    metadata["pop"]
    .sample(frac=1.0, random_state=SEED)
    .groupby(metadata["pop"])
    .apply(lambda x: x.iloc[: n_examples_ct[x.name - 1]].index.values)
)
# selected_indices = np.ar
indices = np.concatenate(selected_indices.values) - 1
print(selected_indices.apply(lambda x: len(x)))

dataset = GeneExpressionDataset()
dataset.populate_from_data(
    X=x_obs.values[indices],
    batch_indices=batch_info["x"].values[indices],
    labels=metadata["pop"].values[indices],
    cell_types=metadata["pop"].values[indices],
)


label_a = 1
label_b = 2
n_genes = dataset.nb_genes

d_key = "{}{}".format(label_a + 1, label_b + 1)
print(d_key)

is_significant_de = (lfc_info[d_key].abs() >= DELTA).values
lfc_gt = lfc_info[d_key].values

USING OFFSET = 1e-10
[]
1 2 (2000,) (2000,)


/data/yosef2/users/pierreboyeau/conda/envs/DE/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
INFO:scvi.dataset.dataset:Remapping batch_indices to [0,N]
INFO:scvi.dataset.dataset:Remapping labels to [0,N]


Original data distrib:  pop
1    2000
2    2000
3    2000
4    2000
5    2000
Name: pop, dtype: int64
pop
1    2000
2     500
3     100
4    2000
5    2000
Name: pop, dtype: int64
23


In [9]:
y_all = dataset.labels.squeeze()

In [10]:
idx_a = np.where(y_all == 1)[0]
idx_b = np.where(y_all == 2)[0]
print(idx_a.shape, idx_b.shape)

(500,) (100,)


In [11]:
import torch.nn as nn
from tqdm.auto import tqdm
import logging

from experiments_utils import get_eb_full
import plotnine as p9

# Experiments

## scVI-lvm

In [10]:
torch.manual_seed(0)

In [11]:
mdl_iw_kwargs = dict(
    n_latent=10,
    dropout_rate=0.0,
    decoder_dropout_rate=0.0,
    reconstruction_loss="nb",
    dispersion="gene",
    n_layers=1,
    use_batch_norm=False,
    use_weight_norm=False,
    use_layer_norm=True,
    with_activation=nn.ReLU(),
)
trainer_iw_kwargs = dict(
    train_library=True,
    batch_size=1024,
    weight_decay=1e-4,
    k=25,
    loss_type="IWELBO",
    optimizer_type="adam",
    test_indices=[],
)

In [12]:
delta = None

%matplotlib inline

N_EPOCHS = 500

In [13]:

algos = [
    dict(
        name="scVI-lvm + ELBO", 
        trainer_kwargs=dict(loss_type="ELBO", k=1)
    ),
#     dict(
#         name="scVI-lvm + ELBO + LN + NOWD", 
#         trainer_kwargs=dict(loss_type="ELBO", k=1, weight_decay=0.)
#     ),
#     dict(
#         name="scVI-lvm + ELBO + BN", 
#         trainer_kwargs=dict(loss_type="ELBO", k=1),
#         mdl_kwargs=dict(use_batch_norm=True, use_layer_norm=False)
#     ),
#     dict(name="scVI-lvm (fixed LR)", autolr=False),
#     dict(name="scVI-lvm + BN", mdl_kwargs=dict(use_batch_norm=True, use_layer_norm=False)),
    dict(name="scVI-lvm + IWELBO"),
#     dict(name="scVI-lvm + IWELBO", trainer_kwargs=dict(weight_decay=0.)),
#     dict(name="scVI-lvm + DREG + LN", trainer_kwargs=dict(loss_type="IWELBO")),
#     dict(name="scVI-lvm + DREG + LN + NOWD", trainer_kwargs=dict(loss_type="IWELBO", weight_decay=0.)),
#     dict(name="scVI-lvm + BN + NOWD", mdl_kwargs=dict(use_batch_norm=True, use_layer_norm=False), trainer_kwargs=dict(weight_decay=0.)),
#     dict(name="scVI-lvm + IWELBO ++ LN2 + NOWD", mdl_kwargs=dict(use_batch_norm=False, use_layer_normb=True, use_layer_norm=False), trainer_kwargs=dict(weight_decay=0.)),
#     dict(name="scVI-lvm + LN2", mdl_kwargs=dict(use_batch_norm=False, use_layer_normb=True, use_layer_norm=False)),
]

results = pd.DataFrame()
for iterate in range(5):
    for algo in (algos):
        # Overall parameters
        algo = algo.copy()
        algo_name = algo.pop("name")
        autolr = algo.pop("autolr", True)
        print(algo_name)

        # Model parameters
        mdl_params_iter = mdl_iw_kwargs.copy()
        _mdl_params_iter = algo.pop("mdl_kwargs", dict())
        for key, new_val in _mdl_params_iter.items():
            print(key, new_val)
            mdl_params_iter[key] = new_val

        # Trainer parameters
        trainer_params_iter = trainer_iw_kwargs.copy()
        _trainer_params_iter = algo.pop("trainer_kwargs", dict())
        for key, new_val in _trainer_params_iter.items():
            print(key, new_val)
            trainer_params_iter[key] = new_val

        print()

        mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_params_iter)
        trainer_iw = UnsupervisedTrainer(
            model=mdl_iw, gene_dataset=dataset, **trainer_params_iter
        )
#         if not autolr:
        logging.info("Using fixed LR")
        lr_iw = 1e-3
#         else:
#             lr_iw = trainer_iw.find_lr()
#             mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_params_iter)
#             trainer_iw = UnsupervisedTrainer(
#                 model=mdl_iw, gene_dataset=dataset, **trainer_params_iter
#             )
    
        logging.info("Using learning rate {}".format(lr_iw))
        trainer_iw.train(n_epochs=N_EPOCHS, lr=lr_iw)
        mdl_iw.eval()
        
        lls = trainer_iw.train_set.marginal_llb(custom_batch_index=None).mean().item()

        mf_props_eb = get_eb_full(
            trainer=trainer_iw,
            idx_a=idx_a,
            mode_coeff=0.6,
            idx_b=idx_b,
            offset=None,
            delta=None,
            # n_samples=30000,
            n_samples=50000,
            include_lib=False,
            # keeping_k_best_subsamples=20000,
            do_batch_specific="separation",
            posterior_chunks=200,
            do_normalize=True,
            filter_cts=True,
#             coef_truncate=.5,
        )
        
        subres_d = {**mf_props_eb, **dict(lfc_gt=lfc_gt, is_significant_de=is_significant_de)}
        subres = pd.DataFrame(subres_d).assign(algo_name=algo_name, iterate=iterate, iwelbo5000=lls)
        results = results.append(subres, ignore_index=True)
    #     break

results.to_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_iw7.pickle")

INFO:root:Normal parameterization of the library
INFO:root:Scale decoder with Softmax normalization


scVI-lvm + ELBO
loss_type ELBO
k 1



INFO:root:Using fixed LR
INFO:root:Using learning rate 0.001
INFO:root:Unique optim


training: 100%|██████████| 500/500 [07:39<00:00,  1.09it/s]

INFO:root:Using 5000 samples for log-evidence estimation ...


...

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches


torch.Size([1, 39])
torch.Size([25038, 39])
tensor([1])


/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:401: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
INFO:root:ESS: 2223.491624665802
INFO:root:LFC MEAN in batch 1: -0.07170718163251877


Data inferred offsets: 1e-10 2.7267740006209354e-05
using offset: 2.7267740006209354e-05


INFO:root:DELTA VALUE: 0.6499730533375946
INFO:root:Using mode coefficient 0.6


## scPhere-lvm

In [14]:
mdl_sph_kwargs = dict(
    n_genes=dataset.nb_genes,
    n_batches=dataset.n_batches,
    n_latent=11,
    dropout_rate=0.0,
    do_depth_reg=False,
    constant_pxr=True,
    cell_specific_px=False,
    use_batch_norm=False, 
    use_layer_norm=True,
    deep_architecture=False,    
    full=True,
)
trainer_sph_kwargs = dict(
    train_library=True,
    k=25,
    loss_type="IWELBO",
    batch_size=1024,
    weight_decay=1e-4,
    optimizer_type="adam",
    test_indices=[],
)

algos = [
#     dict(name="scPhere-lvm"),
    dict(name="scPhere-lvm", mdl_kwargs=dict(deep_architecture=True)),
    dict(
        name="scPhere-lvm - constant overdispersion", 
        mdl_kwargs=dict(
            deep_architecture=True,
            constant_pxr=False,
            cell_specific_px=True,
        )
    ),
    dict(
        name="scPhere-lvm + ELBO", 
        mdl_kwargs=dict(deep_architecture=True),
        trainer_kwargs=dict(loss_type="ELBO", k=1)
    ),
    dict(
        name="scPhere-lvm - library size", 
        mdl_kwargs=dict(deep_architecture=True, full=False)
    ),
]

from scvi.models import SCSphere, SCSphereFull


results = pd.DataFrame()
for iterate in range(5):
    for algo in (algos):
        # Overall parameters
        print(algo)
        algo = algo.copy()
        algo_name = algo.pop("name")
        autolr = algo.pop("autolr", True)
        print(algo_name)

        # Model parameters
        mdl_params_iter = mdl_sph_kwargs.copy()
        _mdl_params_iter = algo.pop("mdl_kwargs", dict())
        
        for key, new_val in _mdl_params_iter.items():
            print(key, new_val)
            mdl_params_iter[key] = new_val

        # Trainer parameters
        trainer_params_iter = trainer_sph_kwargs.copy()
        _trainer_params_iter = algo.pop("trainer_kwargs", dict())
        for key, new_val in _trainer_params_iter.items():
            print(key, new_val)
            trainer_params_iter[key] = new_val

        print()
        do_full = mdl_params_iter.pop("full", True,)
        if do_full:
            mdl_iw = SCSphereFull(**mdl_params_iter)
        else:
            mdl_iw = SCSphere(**mdl_params_iter)
        trainer_iw = UnsupervisedTrainer(
            model=mdl_iw, gene_dataset=dataset, **trainer_params_iter
        )
#         if not autolr:
        logging.info("Using fixed LR")
        lr_iw = 1e-3
#         else:
#             lr_iw = trainer_iw.find_lr()
#             mdl_iw = SCSphere(**mdl_params_iter)
#             trainer_iw = UnsupervisedTrainer(
#                 model=mdl_iw, gene_dataset=dataset, **trainer_params_iter
#             )
        logging.info("Using learning rate {}".format(lr_iw))
        trainer_iw.train(n_epochs=N_EPOCHS, lr=1e-3)
        mdl_iw.eval()

        mf_props_eb = get_eb_full(
            trainer=trainer_iw,
            idx_a=idx_a,
            mode_coeff=0.6,
            idx_b=idx_b,
            offset=None,
            delta=None,
            n_samples=50000,
            do_batch_specific="separation",
            posterior_chunks=200,
            do_normalize=False,
            filter_cts=True,
        )

        
        subres_d = {**mf_props_eb, **dict(lfc_gt=lfc_gt, is_significant_de=is_significant_de)}
        subres_d["gene_idx"] = np.arange(len(is_significant_de))
        subres = pd.DataFrame(subres_d).assign(algo_name=algo_name, iterate=iterate)

        results = results.append(subres, ignore_index=True)

    #     break

INFO:root:Using Deep architecture ...
INFO:root:Using deep architecture
INFO:root:Using fixed LR
INFO:root:Using learning rate 0.001
INFO:root:Unique optim


{'name': 'scPhere-lvm', 'mdl_kwargs': {'deep_architecture': True}}
scPhere-lvm
deep_architecture True

training: 100%|██████████| 500/500 [09:26<00:00,  1.13s/it]

INFO:root:Pop A 500 & Pop B 100
INFO:root:Using mode separation
INFO:root:Filtering observations: Keeping (219,) cells from original 244 sample size
INFO:root:Using 115 posterior samples per cell
INFO:root:Step 1: Getting posterior samples
INFO:root:Step 2: Compute overall importance weights



None


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches
/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:401: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1, 39])
torch.Size([25038, 39])
tensor([1])


INFO:root:ESS: 1743.1600672471225
INFO:root:LFC MEAN in batch 1: 0.10046479105949402
INFO:root:DELTA VALUE: 0.2908979872935486
INFO:root:Using mode coefficient 0.6


Data inferred offsets: 1e-10 2.3760529711580602e-05
using offset: 2.3760529711580602e-05


In [15]:
results.to_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_sph7_.pickle")

## Features

In [16]:
mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_iw_kwargs)
trainer_iw = UnsupervisedTrainer(
    model=mdl_iw, gene_dataset=dataset, **trainer_iw_kwargs
)
lr_iw = trainer_iw.find_lr()
mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_iw_kwargs)
trainer_iw = UnsupervisedTrainer(
    model=mdl_iw, gene_dataset=dataset, **trainer_iw_kwargs
)
logging.info("Using learning rate {}".format(lr_iw))
trainer_iw.train(n_epochs=500, lr=1e-2)
mdl_iw.eval()
print()

INFO:root:Normal parameterization of the library
INFO:root:Scale decoder with Softmax normalization
INFO:root:Unique optim
INFO:root:Automatic LR choice 0.0004641588833612781
INFO:root:Normal parameterization of the library
INFO:root:Scale decoder with Softmax normalization
INFO:root:Using learning rate 0.0004641588833612781
INFO:root:Unique optim


training: 100%|██████████| 500/500 [08:56<00:00,  1.07s/it]



In [17]:
delta = None

base_features = dict(
    trainer=trainer_iw,
    idx_a=idx_a,
    mode_coeff=0.6,
    idx_b=idx_b,
    offset=None,
    delta=delta,
    n_samples=20000,
    do_batch_specific="separation",
    posterior_chunks=200,
    do_normalize=True,
#     do_normalize=True,
    filter_cts=True,
)

In [18]:
features = [
    dict(name="Reference"),
    dict(name="Reference - outliers", filter_cts=False),
    dict(name="Reference - pseudocounts", offset=1e-12),
    dict(name="Reference - autolr", delta=.8),
]

In [19]:
res_feat = pd.DataFrame()

for iterate in range(5):
    for feat in features:
        feat = feat.copy()
        algo_name = feat.pop("name")
        iter_feats = base_features.copy()
        print("\n\n\n")
        for key, new_val in feat.items():
            print(key, new_val)
            iter_feats[key] = new_val
        mf_props_eb = get_eb_full(
            **iter_feats
        )


        subres_d = {**mf_props_eb, **dict(lfc_gt=lfc_gt, is_significant_de=is_significant_de)}
        subres_d["gene_idx"] = np.arange(len(is_significant_de))
        subres = pd.DataFrame(subres_d).assign(algo_name=algo_name, iterate=iterate)

        res_feat = res_feat.append(subres, ignore_index=True)


res_feat.to_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_features.pickle")

INFO:root:Pop A 500 & Pop B 100
INFO:root:Using mode separation
INFO:root:Filtering observations: Keeping (219,) cells from original 244 sample size
INFO:root:Using 46 posterior samples per cell
INFO:root:Step 1: Getting posterior samples
INFO:root:Step 2: Compute overall importance weights






None


  0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches
/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:401: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
INFO:root:ESS: 3296.0246309690538
INFO:root:Filtering observations: Keeping (50,) cells from original 56 sample size
INFO:root:Using 200 posterior samples per cell
INFO:root:Step 1: Getting posterior samples
INFO:root:Step 2: Compute overall importance weights


torch.Size([1, 219])
torch.Size([10074, 219])
tensor([0])
None


...

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches
/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:401: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
INFO:root:ESS: 2604.676488757165
INFO:root:Filtering observations: Keeping (39,) cells from original 44 sample size
INFO:root:Using 257 posterior samples per cell
INFO:root:Step 1: Getting posterior samples
INFO:root:Step 2: Compute overall importance weights


torch.Size([1, 230])
torch.Size([10120, 230])
tensor([1])
None


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches
/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:401: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
INFO:root:ESS: 661.2949648972105
INFO:root:LFC MEAN in batch 1: -0.0380527526140213


torch.Size([1, 39])
torch.Size([10023, 39])
tensor([1])
Data inferred offsets: 1e-10 1.290798136324156e-05
using offset: 1.290798136324156e-05


# Interpreting results

In [12]:
os.listdir("lfc_estimates_both_rareFIXED5/symsim_ablation_study")

['symsim_bimod_100_500_1e-10_0.8_2000_4',
 'results_sph7.pickle',
 'results_iw6.pickle',
 'results_scvi.pickle',
 'symsim_bimod_100_500_1e-10_0.8_2000_3',
 'results_sph3.pickle',
 'symsim_bimod_100_500_1e-10_0.8_2000_0',
 'symsim_bimod_100_500_1e-10_0.8_2000_2',
 'results_iw3.pickle',
 'results_sph2.pickle',
 'results_features.pickle',
 'symsim_bimod_100_500_1e-10_0.8_2000_1',
 'results_sph6.pickle',
 'results_iw7.pickle',
 'results_iw5.pickle']

In [38]:
# results = pd.read_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_iw7.pickle")
# results = pd.read_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_sph7.pickle")
results = pd.read_pickle("lfc_estimates_both_rareFIXED5/symsim_ablation_study/results_features.pickle")

In [39]:
from experiments_utils import plot_tprfdr_curves, predict_de_genes

gene_idx = (
    results
    .groupby(["algo_name", "iterate"])
    .apply(lambda x: pd.Series(np.arange(len(x))))
    .T
    .unstack()
    .reset_index()
)
gene_idx

rename_cols = dict(
    filt_opt_full_eb_is_de2="de_score",
    filt_opt_full_eb_lfc_estim="lfc_estim",
    filt_opt_full_eb_lfc_std="lfc_std",
    filt_opt_full_eb_lfc_estim2="lfc_estim2",
#     opt_full_eb_is_de2="de_score",
#     opt_full_eb_lfc_estim="lfc_estim",
#     opt_full_eb_lfc_std="lfc_std",
#     opt_full_eb_lfc_estim2="lfc_estim2",
)

res1d = (
    results
    .assign(gene_idx=lambda x: gene_idx.level_2.values)
    .rename(columns=rename_cols).fillna(0.)
)


# preds01 = (
#     results
#     .groupby("algo_name")
#     .apply(lambda x: pd.Series(predict_de_genes(x.de_score.values, desired_fdr=0.1)))
#     .T
#     .stack()
#     .reset_index()
#     .rename(columns={"level_0": "gene_idx", 0: "preds01"})
# )

# preds005 = (
#     results
#     .groupby("algo_name")
#     .apply(lambda x: pd.Series(predict_de_genes(x.de_score.values, desired_fdr=0.05)))
#     .T
#     .stack()
#     .reset_index()
#     .rename(columns={"level_0": "gene_idx", 0: "preds005"})
# )

# preds02 = (
#     results
#     .groupby("algo_name")
#     .apply(lambda x: pd.Series(predict_de_genes(x.de_score.values, desired_fdr=0.2)))
#     .T
#     .stack()
#     .reset_index()
#     .rename(columns={"level_0": "gene_idx", 0: "preds02"})
# )
# preds01
# res1d = res1d.merge(preds01, on=["gene_idx", "algo_name"])
# res1d = res1d.merge(preds02, on=["gene_idx", "algo_name"])
# res1d = res1d.merge(preds005, on=["gene_idx", "algo_name"])
res1d

,de_score,lfc_estim,lfc_std,filt_opt_full_eb_ppos,filt_opt_full_eb_pneg,lfc_estim2,khat_a0,khat_b0,khat_a1,khat_b1,...,opt_full_eb_lfc_std,opt_full_eb_ppos,opt_full_eb_pneg,opt_full_eb_lfc_estim2,filt_full_eb_is_de2,filt_full_eb_lfc_estim,filt_full_eb_lfc_std,filt_full_eb_ppos,filt_full_eb_pneg,filt_full_eb_lfc_estim2
0,0.00000,0.054405,0.171949,0.57975,0.42025,0.008678,0.340707,0.476626,0.553424,0.551720,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000
1,0.00060,-0.227768,0.146223,0.06315,0.93685,-0.199001,0.340707,0.476626,0.553424,0.551720,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000
2,0.01055,0.162581,0.234749,0.76320,0.23680,0.085583,0.340707,0.476626,0.553424,0.551720,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000
3,1.00000,-1.885982,0.237773,0.00000,1.00000,-1.885982,0.340707,0.476626,0.553424,0.551720,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000
4,0.00000,-0.026363,0.143219,0.42275,0.57725,-0.004073,0.340707,0.476626,0.553424,0.551720,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.451896,0.522118,0.496633,0.521388,...,0.0,0.0,0.0,0.0,0.00000,-0.246451,0.093862,0.00045,0.99955,-0.246229
7916,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.451896,0.522118,0.496633,0.521388,...,0.0,0.0,0.0,0.0,0.00070,-0.226437,0.179491,0.11655,0.88345,-0.173655
7917,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.451896,0.522118,0.496633,0.521388,...,0.0,0.0,0.0,0.0,0.00155,-0.259671,0.226709,0.14755,0.85245,-0.183042
7918,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.451896,0.522118,0.496633,0.521388,...,0.0,0.0,0.0,0.0,0.99850,-1.508290,0.234565,0.00000,1.00000,-1.508290


### Posterior expectations

In [40]:
import plotnine as p9

from scvi_utils import true_fdr, posterior_expected_fdr

def get_fdrs(my_df):
    true_fdr_arr = true_fdr(y_true=my_df.is_significant_de.values, y_pred=my_df.de_score.values)
    pe_fdr_arr, y_decision_rule = posterior_expected_fdr(y_pred=my_df.de_score.values)
    return pd.DataFrame(dict(PEFDR=pe_fdr_arr, FDR=true_fdr_arr))

fdrs_df = res1d.groupby(["algo_name", "iterate"]).apply(get_fdrs).reset_index().rename(columns={"level_2": "gene_idx"})
fdrs_df

%matplotlib inline

(
    p9.ggplot(fdrs_df)
    + p9.geom_line(p9.aes(x="gene_idx", y="PEFDR"))
    + p9.geom_line(p9.aes(x="gene_idx", y="FDR"))
    + p9.facet_wrap("algo_name")
#     + p9.theme(figure_size=(15, 5))
)

fdr_info =  (
    fdrs_df
    .groupby(["algo_name", "iterate"])
    .apply(lambda x: np.median(np.abs(x.FDR - x.PEFDR)))
    .reset_index()
    .rename(columns={0: "FDR MAE"})
    .groupby("algo_name")
    ["FDR MAE"].mean()
)
# fdr_info.name = "FDR MAE"
fdr_info

algo_name
Reference                   0.027676
Reference - autolr          0.318533
Reference - outliers        0.318533
Reference - pseudocounts    0.029191
Name: FDR MAE, dtype: float64

### Gene rankings

In [41]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from scipy.stats import pearsonr

# def prerec(my_df):
    

ranks_info = (
    res1d
    .groupby(["algo_name", "iterate"])
#     .apply(lambda x: pearsonr(x.lfc_gt, x.lfc_estim)[0])
#     .apply(lambda x: np.linalg.norm(x.lfc_gt - x.lfc_estim))
    .apply(lambda x: ((x.lfc_estim - x.lfc_gt) ** 2).mean())
#     .apply(lambda x: roc_auc_score(x.is_significant_de, x.de_score))
    .reset_index()
    .rename(columns={0: "Ranking score"})
    .groupby("algo_name")
    ["Ranking score"]
    .mean()
)
ranks_info

algo_name
Reference                   0.101857
Reference - autolr          0.728506
Reference - outliers        0.728506
Reference - pseudocounts    0.127879
Name: Ranking score, dtype: float64

### Stability

In [42]:
stab_info = (
    res1d
    .groupby(["algo_name", "iterate"])
    .apply(lambda x: np.mean(x[["khat_a0", "khat_b0", "khat_a1", "khat_b1"]].values))
    .reset_index()
    .rename(columns={0: "Ranking score"})
    .groupby("algo_name")
    ["Ranking score"]
    .mean()
)
stab_info.name = "khat"
stab_info

algo_name
Reference                   0.504135
Reference - autolr          0.514178
Reference - outliers        0.542678
Reference - pseudocounts    0.541821
Name: khat, dtype: float64

### NCs

In [43]:
null_genes = np.where(~is_significant_de)[0]
null_info = (
    res1d
    .groupby(["algo_name", "iterate"])
    .apply(lambda x: np.mean(x.loc[lambda y: y.gene_idx.isin(null_genes)].lfc_estim**2))
    .reset_index()
    .rename(columns={0: "Ranking score"})
    .groupby("algo_name")
    ["Ranking score"]
    .mean()
)
null_info.name = "NC"
null_info

algo_name
Reference                   0.079291
Reference - autolr          0.000000
Reference - outliers        0.000000
Reference - pseudocounts    0.148819
Name: NC, dtype: float64

### Aggregating results

In [44]:
agg_res = (
    pd.concat([
        fdr_info, 
        ranks_info, 
        stab_info,
        null_info
    ], axis=1)
    .assign(new_index=lambda x: x.index.str.replace("Reference", "Ref"))
    .set_index("new_index")
)
agg_res

,FDR MAE,Ranking score,khat,NC
new_index,,,,
Ref,0.027676,0.101857,0.504135,0.079291
Ref - autolr,0.318533,0.728506,0.514178,0.000000
Ref - outliers,0.318533,0.728506,0.542678,0.000000
Ref - pseudocounts,0.029191,0.127879,0.541821,0.148819


In [45]:
for row in agg_res.iterrows():
    row_vals = ["{0:.4f}".format(va) for va in row[-1]]
    print(row[0] + " & "+ " & ".join(row_vals) + r" \\")

Ref & 0.0277 & 0.1019 & 0.5041 & 0.0793 \\
Ref - autolr & 0.3185 & 0.7285 & 0.5142 & 0.0000 \\
Ref - outliers & 0.3185 & 0.7285 & 0.5427 & 0.0000 \\
Ref - pseudocounts & 0.0292 & 0.1279 & 0.5418 & 0.1488 \\


In [111]:
agg_res1d = (
    agg_res
    .unstack()
    .reset_index()
    .rename(columns={"level_0": "Metric", "algo_name": "Algorithm", 0: "Values"})
    .assign(val_text=lambda x: x.Values.round(2).astype(str))
)
agg_res1d

,Metric,new_index,Values,val_text
0,FDR MAE,scPhere-lvm,5.717934e-02,0.06
1,FDR MAE,scPhere-lvm + ELBO,3.185335e-01,0.32
2,FDR MAE,scPhere-lvm - constant overdispersion,2.996261e-02,0.03
3,FDR MAE,scPhere-lvm - library size,1.672428e-01,0.17
4,Ranking score,scPhere-lvm,1.117339e-01,0.11
5,Ranking score,scPhere-lvm + ELBO,7.282319e-01,0.73
6,Ranking score,scPhere-lvm - constant overdispersion,2.687753e-01,0.27
7,Ranking score,scPhere-lvm - library size,3.044722e-01,0.3
8,khat,scPhere-lvm,8.979503e-02,0.09
9,khat,scPhere-lvm + ELBO,-1.041820e-01,-0.1


In [ ]:
# from scipy.stats import pearsonr

# (
#     res1d
#     .groupby("algo_name")
#     .apply(lambda x: pearsonr(x.lfc_gt, x.lfc_estim2)[0])
    
# )

In [ ]:
# res1d.loc[lambda x: x.algo_name == "scVI-lvm + layer norm"].lfc_estim2.mean()

In [ ]:
(
    p9.ggplot(res1d)
    + p9.geom_point(p9.aes(x="lfc_gt", y="lfc_estim2"))
    + p9.facet_wrap("algo_name")
)